## Семинар 10: Нейросетевые модели поиска. Часть II.

В этом семинаре мы:
- вспомним основы pytorch;
- познакомимся с библиотекой [transformers](https://huggingface.co/docs/transformers/index);
- загрузим претрейн XLM-RoBERTa и дообучим под задачу ранжирования.

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# %pip install --upgrade pip
# %pip install -r requirements.txt

### Torch: recap

В этой секции мы вспомним основы библиотеки `torch`.

Самостоятельно вы можете посмотреть [небольшой обзор](https://kharshit.github.io/blog/2021/12/03/pytorch-basics-tutorial) базовых понятий.

Как установить pytorch: https://pytorch.org/get-started/locally/

In [3]:
import torch

Основа торча - это `torch.Tensor` (аналог `np.ndarray`), который поддерживает работу на CPU / GPU и автодифференцирование (autograd).

In [4]:
# C, H, W
a = torch.Tensor(size=(3, 28, 28))
print(a.dtype, a.type(), a.shape)
# a.reshape()
print(a.view(3, -1).shape)

torch.float32 torch.FloatTensor torch.Size([3, 28, 28])
torch.Size([3, 784])


Можем конвертировать np в torch и обратно:

In [5]:
b = torch.ones(2, 2)
# tensor -> np array
b = b.numpy()
print(type(b))

# np array -> tensor
b = torch.tensor(b)  # or torch.from_numpy(b)
print(type(b))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


Можем положить тензора на GPU и выполнять любые операции:

In [6]:
# check if CUDA available
print(torch.cuda.is_available())
# check if tensor on GPU
print(b.is_cuda)
# move tensor to GPU: defaults to gpu:0
print(b.cuda()) # or to.device('cuda')
# move tensor to CPU
print(b.cpu()) # or to.device('cpu')
# check tensor device
print(b.device)

True
False
tensor([[1., 1.],
        [1., 1.]], device='cuda:0')
tensor([[1., 1.],
        [1., 1.]])
cpu


Torch поддерживает автоматический расчет градиентов через autograd:

In [7]:
import time

x = torch.randn(2, 2, requires_grad=True)
print(f'x.grad: {x.grad}')
print(f'x.grad_fn: {x.grad_fn}')

# Строим граф
y = x**2
print(f'y.grad_fn: {y.grad_fn}')

print('#' * 60)

# Рассчитываем градиенты
z = y.mean()
z.backward()

print(f'z.grad: {z.grad}')
time.sleep(0.5)
print(f'z.grad_fn: {z.grad_fn}')

print(f'y.grad: {y.grad}')
time.sleep(0.5)
print(f'y.grad_fn: {y.grad_fn}')

print(f'x.grad: {x.grad}')
time.sleep(0.5)
print(f'x.grad_fn: {x.grad_fn}')


x.grad: None
x.grad_fn: None
y.grad_fn: <PowBackward0 object at 0x7f2440479700>
############################################################
z.grad: None


/tmp/ipykernel_4174/1647573470.py:17: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(f'z.grad: {z.grad}')


z.grad_fn: <MeanBackward0 object at 0x7f244047d490>
y.grad: None


/tmp/ipykernel_4174/1647573470.py:21: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(f'y.grad: {y.grad}')


y.grad_fn: <PowBackward0 object at 0x7f2440462c10>
x.grad: tensor([[-0.2092,  0.1500],
        [-0.5400,  0.2252]])
x.grad_fn: None


Также нам понадобится опция отключения дифференцирования:

In [8]:
print(f"{x.requires_grad=}")
print(f"{(x ** 2).requires_grad=}")

with torch.no_grad():
    print(f"{(x ** 2).requires_grad=}")

y = x.detach()
print(f"{x.requires_grad=}")
print(f"{y.requires_grad=}")

# best way to copy a tensor
# y = x.detach().clone()

x.requires_grad=True
(x ** 2).requires_grad=True
(x ** 2).requires_grad=False
x.requires_grad=True
y.requires_grad=False


### Подготовка данных

В этот раз мы снова будем работать с датасетом MS MARCO. Он по-прежнему содержит набор запросов (=сессий) и соответствующие пассажи (=документы). Предобработка не изменилась, поэтому просто скачаем готовый датасет.

In [9]:
import os
import numpy as np
import pandas as pd

from IPython.display import clear_output
from tqdm import tqdm

In [10]:
DATA_DIR = os.path.abspath("./data")
if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)

In [11]:
# !source download_data.sh https://cloud.mail.ru/public/siz9/gg6FnPs1v ./data/ms_marco_tokenized.pck

In [12]:
data = pd.read_pickle(os.path.join(DATA_DIR, "ms_marco_tokenized.pck"))

Посмотрим на получившийся датасет. Помним, что для каждого запроса в среднем 1 релевантный и 29 нерелевантных документов.

In [13]:
data.head()

qid                       query  \
0    0  where is whitemarsh island   
1    0  where is whitemarsh island   
2    0  where is whitemarsh island   
3    0  where is whitemarsh island   
4    0  where is whitemarsh island   

                                                 doc  label  \
0  Whitemarsh Island, Georgia. Whitemarsh Island ...    1.0   
1  the strategy of island hopping was used by the...    0.0   
2  For the island near Dunedin, see White Island,...    0.0   
3  Jekyll Island, at 5,700 acres, is the smallest...    0.0   
4  Sibu Island. A scuba diver at Sibu Island. Sib...    0.0   

                      query_tokens  \
0  [where, is, whitemarsh, island]   
1  [where, is, whitemarsh, island]   
2  [where, is, whitemarsh, island]   
3  [where, is, whitemarsh, island]   
4  [where, is, whitemarsh, island]   

                                          doc_tokens  
0  [whitemarsh, island, ,, georgia, ., whitemarsh...  
1  [the, strategy, of, island, hopping, was, used...  
2  [for, the, island, near, dunedin, ,, see, whit...  
3  [jekyll, island, ,, at, 5, ,, 700, acres, ,, i...  
4  [sibu, island, ., a, scuba, diver, at, sibu, i...

Разделим датасет на train / val / test. Разделяем с группировкой по сессиям (запросам).

In [14]:
TEST_SIZE = 3_000
test_data = data[(400_000  - TEST_SIZE < data['qid']) & (data['qid'] <= 400_000)]
val_data = data[(400_000 - 2 * TEST_SIZE < data['qid']) & (data['qid'] <= 400_000 - TEST_SIZE)]
train_data = data[data['qid'] <= 400_000 - 2 * TEST_SIZE]

### Задача и метрика

Будем решать задачу переранжирования текстовых пассажей для запросов.

Для каждого запроса есть набор релевантных и не релевантных пассажей.

Требуется отранжировать пассажи относительно запроса, чтобы релевантный пассаж стоял выше нерелевантных.

Как и в прошлом семинаре, будем использовать метрику [Mean Reciprocal Rank](https://www.evidentlyai.com/ranking-metrics/mean-reciprocal-rank-mrr) (MRR). Она определяется так:

$$ MRR = \frac{1}{|Q|} \sum_{q_i} \frac{1}{rank_{i}},$$

где $ rank_i $ - позиция __первого релевантного__ док-та для запроса $q_i$, $ |Q| $ - кол-во запросов в выборке.

In [15]:
from torchmetrics.retrieval import RetrievalMRR

# Подробнее про реализацию:
# https://github.com/Lightning-AI/torchmetrics/blob/master/src/torchmetrics/functional/retrieval/reciprocal_rank.py
mrr = RetrievalMRR(top_k=10)

def MRR(preds, target, qids):
    assert isinstance(preds, np.ndarray)
    assert isinstance(target, np.ndarray)
    assert isinstance(qids, np.ndarray)
    score = mrr(torch.Tensor(preds), torch.Tensor(target), indexes=torch.LongTensor(qids - min(qids)))
    return score.item()

In [16]:
results = {}

test_doc_texts, test_doc_tokens = test_data[["doc", "doc_tokens"]].values.T
test_query_texts, test_query_tokens = test_data.drop_duplicates(subset=["query"])[["query", "query_tokens"]].values.T
test_query_idx = test_data["qid"].factorize()[0] # индексы сессий понадобятся ниже

#### Бейзлайн: Random

Измерим качество случайного предсказания релевантности:

In [17]:
np.random.seed(42)
results["random"] =  MRR(np.random.random(len(test_data)), test_data['label'].values, test_data['qid'].values)
results["random"]

0.09801560640335083

#### Бейзлайн: BM25

Теперь применим алгоритм BM25. До появления трансформеров это был стабильно хороший бейзлайн в задаче ранжирования.

In [18]:
# Код с прошлого семинара

import multiprocessing as mp
from rank_bm25 import BM25Okapi


bm25 = BM25Okapi(list(test_doc_tokens))

def get_bm25_scores(args):
    q_text, q_tokens = args
    doc_scores = bm25.get_scores(q_tokens)
    mask = test_data['query'] == q_text
    return np.where(mask, doc_scores, 0)


cpu_count = mp.cpu_count()
gen = zip(test_query_texts, test_query_tokens)
with mp.Pool(cpu_count) as pool:
    doc_scores = list(tqdm(pool.imap(get_bm25_scores, gen), total=len(test_query_texts)))
bm25_preds = np.sum(doc_scores, axis=0)

results["bm25"] = MRR(bm25_preds, test_data['label'].values, test_data['qid'].values)
results["bm25"]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:49<00:00, 60.88it/s]


0.6004154682159424

In [19]:
# Также добавим результаты для простых эмбеддингов из прошлого семинара:
results["word2vec"] = 0.22002
results["fasttext"] = 0.24425

### Библиотека transformers

In [20]:
from sklearn.metrics import roc_auc_score
from transformers import AutoTokenizer, AutoModel
from torch import nn
from torch.utils.data import Dataset, DataLoader

Возьмем в качестве предобученной модели [xlm-roberta-base](https://huggingface.co/FacebookAI/xlm-roberta-base). Загрузим модель и токенизатор с помощью generic классов:

In [21]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', cache_dir=DATA_DIR)
model = AutoModel.from_pretrained("xlm-roberta-base", cache_dir=DATA_DIR)

/home/stanislav.petrov/.venvs/student10/lib64/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### Про токенизаторы

Пока оставим модель и посмотрим внимательнее на токенизатор:

In [22]:
tokenizer

XLMRobertaTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

Приготовим семпл, чтобы на его примере разобраться в интерфейсе:

In [23]:
sample_query = "IV КРОССОВОК REEBOK МУЖСКОЙ ANSWER".lower()
sample_title = "РОССИЯ БЕСПЛАТНЫЙ ЦЕНА ОТЗЫВ REEBOK V55619 МУЖСКОЙ FOOTBOX КУПИТЬ STEPOVER ПРИМЕРКА IV АРТИКУЛ ИНТЕРНЕТ ДОСТАВКА КРОССОВОК ANSWER МАГАЗИН".lower()

In [24]:
# Разбиваем текст на токены (подслова)
tokenizer.tokenize(sample_query)

['▁i', 'v', '▁крос', 'сов', 'ок', '▁re', 'e', 'bok', '▁муж', 'ской', '▁answer']

In [25]:
encoded_input = tokenizer(sample_query, return_tensors='pt')
encoded_input

{'input_ids': tensor([[     0,     17,    334, 204090,  38920,   2297,    456,     13,  12720,
          30300,   5902,  35166,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [26]:
encoded_input = tokenizer(sample_query, sample_title, return_tensors='pt')
encoded_input

{'input_ids': tensor([[     0,     17,    334, 204090,  38920,   2297,    456,     13,  12720,
          30300,   5902,  35166,      2,      2,  86856,  31126,  11271,  33681,
           2192,  21013, 100414,    456,     13,  12720,     81, 163406,   2947,
          30300,   5902,  57616,  11728,  78297,  29954,   5465,  12049,    415,
             17,    334, 234764,   9727,  86478, 204090,  38920,   2297,  35166,
          21246,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [27]:
tokenizer.decode(encoded_input['input_ids'].view(-1).tolist())

'<s> iv кроссовок reebok мужской answer</s></s> россия бесплатный цена отзыв reebok v55619 мужской footbox купить stepover примерка iv артикул интернет доставка кроссовок answer магазин</s>'

Токены с id = 0 и id = 2 -- специальные токены начала и конца последовательности.

In [28]:
tokenizer.decode([0, 2])

'<s></s>'

Специальные токены бывают разные:

In [29]:
# tokenizer.special_tokens_map

В токенизаторе хранится словарик вида токен -- id токена:

In [30]:
# tokenizer.vocab

Можно токенизировать сразу батч из нескольких строк. В таком случае необходимо указать логику выравнивания последовательностей внутри батча: они должны быть одинаковой длины.

In [31]:
encoded_input = tokenizer([sample_query, sample_title], return_tensors='pt', padding=True, truncation=True)
encoded_input

{'input_ids': tensor([[     0,     17,    334, 204090,  38920,   2297,    456,     13,  12720,
          30300,   5902,  35166,      2,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1],
        [     0,  86856,  31126,  11271,  33681,   2192,  21013, 100414,    456,
             13,  12720,     81, 163406,   2947,  30300,   5902,  57616,  11728,
          78297,  29954,   5465,  12049,    415,     17,    334, 234764,   9727,
          86478, 204090,  38920,   2297,  35166,  21246,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

#### Про модели

Теперь вернемся к моделям. Как выглядит наша модель?

In [32]:
# model

In [33]:
# model.encoder.layer[0]

In [34]:
# for name, par in model.named_parameters():
#     print(name)

In [35]:
# model.config

Выше мы получили батч длиной 2. Сделаем прямой проход через модель и посмотрим на ее выход.

In [36]:
# forward pass
output = model(**encoded_input)
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0910,  0.0780,  0.0613,  ..., -0.0556,  0.0469, -0.0158],
         [-0.0765, -0.0358, -0.0316,  ...,  0.3345,  0.0167, -0.1170],
         [-0.0198, -0.0243, -0.0061,  ...,  0.0950, -0.0217, -0.0441],
         ...,
         [ 0.0318,  0.0126,  0.0411,  ..., -0.0326, -0.0391, -0.0561],
         [ 0.0318,  0.0126,  0.0411,  ..., -0.0326, -0.0391, -0.0561],
         [ 0.0318,  0.0126,  0.0411,  ..., -0.0326, -0.0391, -0.0561]],

        [[ 0.0660,  0.0676,  0.0653,  ..., -0.0466,  0.0372,  0.0063],
         [-0.1194,  0.0219, -0.0149,  ...,  0.0499, -0.0863,  0.1584],
         [-0.0698,  0.0572,  0.0582,  ..., -0.0559, -0.0245,  0.0068],
         ...,
         [-0.1252, -0.0771,  0.0626,  ...,  0.0576, -0.0283,  0.0953],
         [-0.0365,  0.0628,  0.0632,  ..., -0.1079, -0.0211,  0.0636],
         [ 0.0506,  0.0556,  0.0053,  ..., -0.1313, -0.0339,  0.0464]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [37]:
vars(output).keys()

dict_keys(['last_hidden_state', 'pooler_output', 'hidden_states', 'past_key_values', 'attentions', 'cross_attentions'])

In [38]:
# выходы последнего слоя
output.last_hidden_state

tensor([[[ 0.0910,  0.0780,  0.0613,  ..., -0.0556,  0.0469, -0.0158],
         [-0.0765, -0.0358, -0.0316,  ...,  0.3345,  0.0167, -0.1170],
         [-0.0198, -0.0243, -0.0061,  ...,  0.0950, -0.0217, -0.0441],
         ...,
         [ 0.0318,  0.0126,  0.0411,  ..., -0.0326, -0.0391, -0.0561],
         [ 0.0318,  0.0126,  0.0411,  ..., -0.0326, -0.0391, -0.0561],
         [ 0.0318,  0.0126,  0.0411,  ..., -0.0326, -0.0391, -0.0561]],

        [[ 0.0660,  0.0676,  0.0653,  ..., -0.0466,  0.0372,  0.0063],
         [-0.1194,  0.0219, -0.0149,  ...,  0.0499, -0.0863,  0.1584],
         [-0.0698,  0.0572,  0.0582,  ..., -0.0559, -0.0245,  0.0068],
         ...,
         [-0.1252, -0.0771,  0.0626,  ...,  0.0576, -0.0283,  0.0953],
         [-0.0365,  0.0628,  0.0632,  ..., -0.1079, -0.0211,  0.0636],
         [ 0.0506,  0.0556,  0.0053,  ..., -0.1313, -0.0339,  0.0464]]],
       grad_fn=<NativeLayerNormBackward0>)

In [39]:
output.last_hidden_state.shape

torch.Size([2, 34, 768])

In [40]:
output.last_hidden_state[:, 0, :]

tensor([[ 0.0910,  0.0780,  0.0613,  ..., -0.0556,  0.0469, -0.0158],
        [ 0.0660,  0.0676,  0.0653,  ..., -0.0466,  0.0372,  0.0063]],
       grad_fn=<SliceBackward0>)

Получили тензор с размерностями (batch_size=2, sequence_len=34, hidden_size=768).

На 0-ой позиции в каждой последвотельности стоит эмбеддинг токена `[CLS]`.

### Трансформер для ранжирования

Соберем cross-encoder для ранжирования:

In [41]:
class RankBert(nn.Module):
    def __init__(self, train_layers_count=2, emb_size=1):
        super(RankBert, self).__init__()

        self.bert = AutoModel.from_pretrained("xlm-roberta-base")
        self.config = self.bert.config

        # freeze all the layers without bias and LN
        for name, par in self.bert.named_parameters():
            if 'bias' in name or 'LayerNorm' in name:
                continue
            par.requires_grad = False

        # unfreeze some of the layers
        layer_count = self.config.num_hidden_layers
        for i in range(train_layers_count):
            for par in self.bert.encoder.layer[layer_count - 1 - i].parameters():
                par.requires_grad = True

        # map cls token embedding to:
        # - relevance score if emb_size = 1
        # - lower dimension emb if emb_size != 1
        self.head = nn.Linear(self.config.hidden_size, emb_size)

        self.is_cross_encoder = emb_size == 1

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        x = self.bert(input_ids=input_ids,
                      token_type_ids=token_type_ids,
                      attention_mask=attention_mask
                      )[0][:, 0, :] #hidden_state of [CLS]
        x = self.head(x)
        return x

Как бы выглядел bi-encoder вариант переранжироващика:

In [42]:
cross_encoder = RankBert(emb_size=1)
bi_encoder = RankBert(emb_size=64)

Но для такой модели надо подавать данные в другом формате:


- должно быть 2 отдельных тензора для токенов запроса и документа
- bi-encoder: preds = (model(q_tokens) * model(doc_tokens)).sum(-1)
- cross-encoder: preds = model(q_doc_tokens)

### Подготовка данных для обучения

#### torch.utils.data.Dataset

In [43]:
class RankDataset(Dataset):
    def __init__(self, data, neg_p=1.0):
        self.neg_p = neg_p
        if self.neg_p < 1.:
            self.data = pd.concat([data[data['label'] == 1],
                                   data[data['label'] == 0].sample(frac=self.neg_p)])
        else:
            self.data = data

    def __getitem__(self, index):
        query, text, label = self.data.iloc[index, [1, 2, 3]]

        return [query.lower(), text.lower()], label

    def __len__(self):
        return len(self.data)

In [44]:
dataset_train = RankDataset(train_data, neg_p=0.3)
dataset_valid = RankDataset(val_data, neg_p=1.)

print(f"{len(dataset_train)=}")
print(f"{len(dataset_valid)=}")

len(dataset_train)=3934724
len(dataset_valid)=92431


In [45]:
dataset_train[0]

(['where is whitemarsh island',
  'whitemarsh island, georgia. whitemarsh island (pronounced wit-marsh) is a census-designated place (cdp) in chatham county, georgia, united states. the population was 6,792 at the 2010 census. it is part of the savannah metropolitan statistical area. the communities of whitemarsh island are a relatively affluent suburb of savannah.'],
 1.0)

In [46]:
dataset_valid[1]

(['what is the common law',
  'â§ 102 sources of international law(1) a rule of international law is one that has been accepted as such by the international community of states (a) in the form of customary law; (b) by international agreement; or (c) by derivation from general principles common to the major legal systems of the world.'],
 0.0)

#### torch.utils.data.Dataloader

In [47]:
texts = dataset_valid[0][0]
print(texts)
x = tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors='pt')
x

['what is the common law', 'the common law can briefly be described as the part of english law that is derived from custom and judicial precedent, and is distinct from statutory law, equity law, and ecclesiastical law; or, in the u.s. jurisdiction, the body of english law as adopted and adapted by the different states.']


{'input_ids': tensor([[     0,   2367,     83,     70,  39210,  27165,      2,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1],
        [     0,     70,  39210,  27165,    831,  59335,    538,    186, 151552,
            237,     70,   2831,    111, 175457,  27165,    450,     83,  16406,
           4126,   1295, 114122,    136,  80209, 123132,      4,    136,     83,
         117781,   1295,  33908,  31667,  27165,      4,  60715,    939,  27165,
              4,    136, 230907,    141,  27165,     74,    707,      4,     

In [48]:
def compose_batch(batch):
    texts = [x for x, _ in batch]
    ys = torch.tensor([y for _, y in batch]).float()
    tokens = tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors='pt')
    return tokens, ys

In [49]:
example_dataloader = DataLoader(dataset_train, shuffle=False, batch_size=128, collate_fn=compose_batch, num_workers=0)

In [50]:
sample_batch = next(iter(example_dataloader))

In [51]:
sample_batch[0]["input_ids"]

tensor([[     0,   7440,     83,  ...,      4, 186796,      2],
        [     0,   7440,     83,  ...,     70,  16128,      2],
        [     0,   7440,     98,  ...,   7154,      5,      2],
        ...,
        [     0,   2750,    509,  ...,      1,      1,      1],
        [     0,   2750,    509,  ...,     71,    450,      2],
        [     0,   2750,    509,  ...,   5824,  47251,      2]])

Детокенизируем элементы батча

In [52]:
tokenizer.decode(sample_batch[0]['input_ids'][0])

'<s> where is whitemarsh island</s></s> whitemarsh island, georgia. whitemarsh island (pronounced wit-marsh) is a census-designated place (cdp) in chatham county, georgia, united states. the population was 6,792</s>'

In [53]:
tokenizer.decode(sample_batch[0]['input_ids'][1])

'<s> where is your perineum</s></s> that part of the floor of the pelvis that lies between the tops of the thighs. in the male, the perineum lies between the anus and the scrotum. in the female, it includes the external genitalia. the area</s>'

In [54]:
train_dataloader = DataLoader(dataset_train, shuffle=True, batch_size=128, collate_fn=compose_batch, num_workers=16)
validation_dataloader = DataLoader(dataset_valid, shuffle=False, batch_size=512, collate_fn=compose_batch, num_workers=8)

In [55]:
len(dataset_train), len(train_dataloader) * 128

(3934724, 3934848)

In [56]:
len(train_dataloader), len(validation_dataloader)

(30741, 181)

### Обучение

##### Инициализируем модель

In [57]:
model = RankBert(train_layers_count=2)
model.is_cross_encoder

True

In [58]:
!nvidia-smi | head

Thu Dec  5 15:05:13 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   23C    P8     8W / 250W |    482MiB / 11264MiB |      0%      Default |


In [59]:
# Кладем модель на гпу
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

RankBert(
  (bert): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

In [60]:
!nvidia-smi | head

Thu Dec  5 15:05:20 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   26C    P2    57W / 250W |   1596MiB / 11264MiB |      0%      Default |


Видим, что модель занимает на гпушке ~1.5 Gb.

##### Конфиг и tensorboard

In [61]:
# !mkdir -p data/cross_encоder_checkpoint

In [62]:
from torch.utils.tensorboard import SummaryWriter

class config:
    EPOCHS = 1
    LR = 1e-4
    WD = 0.01
    MAX_TRAIN_STEPS = 2000
    LOG_INTERVAL = 250
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    SAVE_DIR = "data/cross_encоder_checkpoint"

writer = SummaryWriter('data/cross_encоder_checkpoint/ms_marco')
loss_fn = nn.BCEWithLogitsLoss()

In [63]:
#!rm -rf cross_encоder_checkpoint

Дока про запуск tensorboard с pytorch: https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html

Чтобы запустить для нашего примера, нужно исполнить следующую команду в командой строке из-под virtualenv окружения:

In [64]:
# tensorboard --logdir=./cross_encоder_checkpoint/ --port=9999
# ... и открыть появившуюся ссылку

##### Шедулер и оптимизатор

In [65]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.LR,
    weight_decay=config.WD
)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    pct_start=0.1,
    max_lr=config.LR,
    epochs=config.EPOCHS,
    steps_per_epoch=len(train_dataloader)
)

Оптимизация в torch: https://pytorch.org/docs/stable/optim.html

Оптимизатор Adam в torch: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html

OneCycle шедулер: https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.OneCycleLR.html#torch.optim.lr_scheduler.OneCycleLR

##### Циклы обучения

In [66]:
# Инстанс модели и данные должны находиться на одном девайсе для прямого / обратного прохода.
def move_batch_to_device(batch, device):
    batch_x, y = batch
    for key in batch_x:
        batch_x[key] = batch_x[key].to(device)
    y = y.to(device)
    return batch_x, y

In [67]:
def validate_one_epoch(epoch_idx, tb_writer):
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    preds = []
    running_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(validation_dataloader, desc="Validating"):
            batch_x, y = move_batch_to_device(batch, config.DEVICE)

            voutputs = model(**batch_x)
            preds.append(voutputs.cpu())

            running_loss += loss_fn(voutputs, y.unsqueeze(-1))

    preds = torch.cat(preds).numpy().squeeze()
    avg_loss = running_loss / len(validation_dataloader)

    # Compute validation auc
    auc = roc_auc_score(val_data['label'].values, preds, labels=np.array([0, 1]))

    # Compute validation mrr
    val_mrr = MRR(preds, val_data['label'].values, val_data['qid'].values)

    # Report metrics & losses: tensorboard
    tb_writer.add_scalar('validation/mrr', val_mrr, epoch_idx)
    tb_writer.add_scalar('validation/loss', avg_loss, epoch_idx)
    tb_writer.add_scalar('validation/auc', auc, epoch_idx)

    return val_mrr, avg_loss, auc

In [ ]:
def train_one_epoch(epoch_idx, tb_writer, max_steps=None, device=config.DEVICE):
    model.train()

    running_loss, running_auc = 0., 0.

    # Here, we use enumerate(training_loader) instead of iter(training_loader),
    # so that we can track the batch index and do some intra-epoch reporting
    max_steps = len(train_dataloader) if max_steps is None else max_steps
    pbar = tqdm(enumerate(train_dataloader), total=max_steps, desc=f"Train")
    for batch_idx, batch in pbar:
        if batch_idx == max_steps:
            break
        # Every data instance is an input + label pair
        batch_x, y = move_batch_to_device(batch, device)

        # Zero gradients for every batch
        optimizer.zero_grad()

        # Make predictions with forward pass
        outputs = model(**batch_x)

        # Compute loss and gradients
        loss = loss_fn(outputs, y.unsqueeze(-1))
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        scheduler.step()

        # Update metrics
        running_loss += loss.item()
        y = y.int().cpu().numpy()
        inbatch_auc = roc_auc_score(y, outputs.detach().cpu().numpy(), labels=np.array([0, 1])) if y.max() == 1 else 1
        running_auc += inbatch_auc

        # Report metrics & losses: locally
        pbar.set_description(f"Train, loss={loss.item():.4f}, auc={inbatch_auc:.4f}")
        pbar.refresh()

        # Report metrics & losses: tensorboard
        tb_x = epoch_idx * max_steps + batch_idx + 1
        tb_writer.add_scalar('lr', scheduler.get_last_lr()[0], tb_x)
        tb_writer.add_scalar('train/inbatch_auc', inbatch_auc, tb_x)
        tb_writer.add_scalar('train/inbatch_loss', loss.item(), tb_x)

    avg_loss = running_loss / max_steps
    avg_auc = running_auc / max_steps

    return avg_loss, avg_auc


In [ ]:
# Одна эпоха займет около 20-25 минут на GTX 1080 Ti

best_vloss = 10**9

for epoch in range(config.EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    train_avg_loss, train_avg_auc = train_one_epoch(epoch, writer, config.MAX_TRAIN_STEPS)
    val_mrr, val_avg_loss, val_avg_vauc = validate_one_epoch(epoch, writer)

    # Report metrics & losses: locally
    print(f'LOSS train {train_avg_loss:.4f} validation {val_avg_loss:.4f}')
    print(f'AUC train {train_avg_auc:.4f} validation {val_avg_vauc:.4f}')
    print(f'MRR validation {val_mrr:.4f}')

    # Save model's checkpoint after epoch
    best_vloss = min(val_avg_loss, best_vloss)

    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_vloss': best_vloss
    }
    torch.save(
        checkpoint,
        f'{config.SAVE_DIR}/ckpt_epoch_{epoch}_loss{best_vloss}.pt'
    )

EPOCH 1:
Validating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 181/181 [04:41<00:00,  1.56s/it]
LOSS train 0.2714 validation 0.1078
AUC train 0.7744 validation 0.9427
MRR validation 0.7720


### Инференс

In [77]:
test_dataset = RankDataset(test_data, neg_p=1.)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=512, collate_fn=compose_batch, num_workers=8)

In [78]:
def get_test_preds(model):
    model.eval()
    y_test = []
    for batch in tqdm(test_dataloader):
        batch_x, _ = move_batch_to_device(batch, config.DEVICE)
        with torch.no_grad():
            preds = model(**batch_x)
            y_test += [preds]

    y_test = torch.cat(y_test).view(-1).cpu().numpy()
    return y_test

In [79]:
y_test = get_test_preds(model.float())
y_test[:5]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 181/181 [04:27<00:00,  1.48s/it]


array([ 0.8141252, -3.4959676, -5.693682 , -4.561694 , -2.0990865],
      dtype=float32)

In [80]:
# make model float16 precision
y_test_half = get_test_preds(model.half())
y_test_half[:5]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 181/181 [02:58<00:00,  1.02it/s]


array([ 0.801, -3.496, -5.695, -4.555, -2.096], dtype=float16)

In [81]:
print("auc (fp32) =", roc_auc_score(test_dataset.data['label'].values, y_test, labels=np.array([0, 1])))
print("auc (fp16) =", roc_auc_score(test_dataset.data['label'].values, y_test_half, labels=np.array([0, 1])))

auc (fp32) = 0.9416710047527433
auc (fp16) = 0.9416661744314


In [82]:
results["xlm-roberta-base"] = MRR(y_test, test_data['label'].values, test_data['qid'].values)
results["xlm-roberta-base (half)"] = MRR(y_test_half, test_data['label'].values, test_data['qid'].values)

In [83]:
for name, value in sorted(results.items(), key=lambda x: -x[1]):
    print(f'{value:.5f}\t', name)

0.77070	 xlm-roberta-base
0.77028	 xlm-roberta-base (half)
0.60042	 bm25
0.24425	 fasttext
0.22002	 word2vec
0.09802	 random


Ура, мы получили решение лучше нашего бейзлайна!

Как улучшать решение:
- пробовать pairwise / listwise лоссы
- разморозить бОльшую часть сети
- учить дольше / больше данных
- попробовать другие претрейны (валидно для английского языка)
- оптимизировать скорость обучения (след. за то же время можно прогнать больше данных)
- расширять контекст / добавлять новые текстовые поля

__Что еще можно посмотреть:__
- Как ускорить обучение с помощью mixed-precision
    - *https://pytorch.org/tutorials/recipes/recipes/amp_recipe.html*
    - https://pytorch.org/docs/stable/notes/amp_examples.html
- Как учить модели на нескольких гпу (можно использовать например на кеггле)
    - https://pytorch.org/tutorials/intermediate/ddp_tutorial.html
    - https://pytorch.org/docs/stable/notes/ddp.html
- Библиотеки для более удобного обучения сетей
    - https://github.com/Lightning-AI/pytorch-lightning (общий случай)
    - https://huggingface.co/docs/transformers/main/en/trainer (трансформеры)
    - https://huggingface.co/docs/transformers/main_classes/trainer    